<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://raw.githubusercontent.com/Arize-ai/phoenix-assets/9e6101d95936f4bd4d390efc9ce646dc6937fb2d/images/socal/github-large-banner-phoenix.jpg" width="1000"/>
        <br>
        <br>
        <a href="https://arize-phoenix.readthedocs.io/projects/evals/en/latest/">Evals Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://arize-ai.slack.com/join/shared_invite/zt-2w57bhem8-hq24MB6u7yE_ZF_ilOYSBw#/shared-invite/email">Community</a>
    </p>
</center>
<h1 align="center">Arize Phoenix Evals 2.0</h1>

Arize Phoenix is a fully open-source AI observability platform. It's designed for experimentation, evaluation, and troubleshooting.

**In this notebook, you will learn how to do the following things using Evals 2.0:**

1. How to evaluate Phoenix project traces.
2. How to improve your custom evaluators using experiments.
3. How to iterate on your application and evals on a realistic example.

<center>
    <h3 align="left">The Evaluation Driven Development Lifecycle</h3>
    <p style="text-align:center">
        <img alt="eval lifecycle" src="https://storage.googleapis.com/arize-phoenix-assets/assets/images/evals_lifecycle.png" width="1000"/>
    </p>
</center>


### Requirements

1. Kaggle API key
2. OpenAI API key
3. A Phoenix instance (cloud or local)


In [ ]:
! uv pip install "arize-phoenix-evals>=2.0.0" "arize-phoenix-client>=1.19.0" arize-phoenix-otel kagglehub openinference-instrumentation-llama_index llama-index numpy pandas --quiet

In [ ]:
! uv pip install openinference-instrumentation-llama_index llama-index

# Dataset Preparation and Setup

We are using a public RAG evaluation dataset. It has two components:

1. A knowledge base of 20 documents of various lengths and sources.
2. 4 question-answer pairs per document.
   - 2 which are not answerable by the document
   - 2 which require a single passage to answer

First, we need to do some data preparation.


In [ ]:
# Download dataset
# Requires a Kaggle API key and username in your environment

import os

import kagglehub

path = kagglehub.dataset_download("samuelmatsuoharris/single-topic-rag-evaluation-dataset")

print("Path to dataset files:", path)
print(os.listdir(path))

In [ ]:
import pandas as pd


def prepare_query_data(path: str) -> pd.DataFrame:
    single_passage_df = pd.read_csv(os.path.join(path, "single_passage_answer_questions.csv"))
    no_answer_df = pd.read_csv(os.path.join(path, "no_answer_questions.csv"))

    # Single-passage questions
    single_passage_processed = pd.DataFrame(
        {
            "document_index": single_passage_df["document_index"],
            "query": single_passage_df["question"],
            "answer": single_passage_df["answer"],
            "query_type": "single_passage",
        }
    )

    # No-answer questions
    no_answer_processed = pd.DataFrame(
        {
            "document_index": no_answer_df["document_index"],
            "query": no_answer_df["question"],
            "answer": "N/A",
            "query_type": "no_answer",
        }
    )

    # Combine all dataframes
    combined_df = pd.concat([single_passage_processed, no_answer_processed], ignore_index=True)

    return combined_df


query_df = prepare_query_data(path)
query_df.sample(5).head()

### Split data into train/test

Split documents into a 60/40 train/test split. We will iterate and experiment on our train set only, leaving the test set for any final comparisons.


In [ ]:
import numpy as np

unique_docs = query_df["document_index"].unique()
print(f"Total unique documents: {len(unique_docs)}")

np.random.seed(42)
sample_size = int(len(unique_docs) * 0.6)
train_docs = np.random.choice(unique_docs, size=sample_size, replace=False)
print(f"Sampled {len(train_docs)} documents ({len(train_docs) / len(unique_docs) * 100:.1f}%)")

# Split queries based on sampled document indices
all_queries = query_df.copy()
train_queries = query_df[query_df["document_index"].isin(train_docs)]
test_queries = query_df[~query_df["document_index"].isin(train_docs)]
print(f"Train queries: {len(train_queries)}, Test queries: {len(test_queries)}")

### Inspect the knowledge base documents


In [ ]:
documents = pd.read_csv(os.path.join(path, "documents.csv"))
documents.head()

### Set Up Phoenix Tracing

This allows us to capture traces not only of our application, but also any evaluations and experiments we do.

You can use either a locally hosted instance of Phoenix or Phoenix Cloud.


In [ ]:
# Set up Phoenix Tracing
from openinference.instrumentation.llama_index import LlamaIndexInstrumentor

from phoenix.otel import register

project_name = "rag-demo"  # project for our application traces
tracer_provider = register(project_name=project_name, verbose=False)
LlamaIndexInstrumentor().instrument(tracer_provider=tracer_provider)

Add your LLM API credentials. Here, we are using OpenAI.


In [ ]:
import os
from getpass import getpass

if not (openai_api_key := os.getenv("OPENAI_API_KEY")):
    openai_api_key = getpass("🔑 Enter your OpenAI API key: ")
os.environ["OPENAI_API_KEY"] = openai_api_key

# Set Up a RAG App using Llama Index

For this demo application, we are building a simple RAG pipeline that has two components:

1. Vector index to retrieve documents
2. LLM to generate responses

For this initial application, let's keep it simple and use the default configuration and prompts from Llama Index.


In [ ]:
import os

from llama_index.core import (
    Document,
    Settings,
    StorageContext,
    VectorStoreIndex,
    load_index_from_storage,
)
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI

index_dir = "llamaindex_store"

# --- Ingest documents ---
if os.path.exists(index_dir):
    storage_context = StorageContext.from_defaults(persist_dir=index_dir)
    index = load_index_from_storage(storage_context)
else:
    # --- Set up the LLM and embedding model ---
    Settings.llm = OpenAI(model="gpt-4o-mini", temperature=0)  # generator
    Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")  # retriever

    kb_docs = []
    for _, row in documents.iterrows():
        doc = Document(
            text=str(row["text"]),
            metadata={"source_url": row["source_url"], "document_index": row["index"]},
            id_=str(row["index"]),
        )
        kb_docs.append(doc)

    index = VectorStoreIndex.from_documents(kb_docs)

# Optional: persist to disk so you can reuse later
index.storage_context.persist(persist_dir=index_dir)

# Create the query engine
query_engine = index.as_query_engine()

Let's test to make sure our RAG system is working:


In [ ]:
query_engine.query("What is data science?")

### Run RAG on Train Set


Let's wrap our query engine so it's easier to run on our dataset.


In [ ]:
from openinference.instrumentation import using_metadata


async def run_rag_with_metadata(example, rag_engine):
    """Ask a question of the knowledge base."""
    metadata = {
        "expected_answer": example["answer"],
        "query_type": example["query_type"],
        "expected_document_index": example["document_index"],
        "split": "test" if example["document_index"] not in train_docs else "train",
    }
    with using_metadata(metadata):
        rag_engine.query(example["query"])

We use the `AsyncExecutor` to run our RAG app on the training dataset with optimal speed.


In [ ]:
# Run application on the train set to get a baseline
from functools import partial

from phoenix.evals.executors import AsyncExecutor
from phoenix.evals.utils import get_tqdm_progress_bar_formatter

executor = AsyncExecutor(
    generation_fn=partial(run_rag_with_metadata, rag_engine=query_engine),
    concurrency=10,  # adjust this as needed
    exit_on_error=True,
    tqdm_bar_format=get_tqdm_progress_bar_formatter("Run RAG"),
)

results, execution_details = await executor.execute(
    [row.to_dict() for _, row in train_queries.iterrows()],
)

# Evaluate the Traces

First, let's go to Phoenix and look at our application traces. Do we observe any issues?

- Is the RAG agent correctly refusing to answer the unanswerable queries?
- Is it retrieving the correct documents?
- Is it hallucinating?

These are common questions we can turn into repeatable evaluations. So let's create a few evaluators for our RAG app and run them on our traces.

**Steps:**

1. Export traces from Phoenix
2. Define evaluators
3. Run evaluators on the trace data
4. Log the evaluation results back up to Phoenix


In [ ]:
from phoenix.client import Client
from phoenix.client.helpers.spans import get_input_output_context, get_retrieved_documents

# fetch the retrieved document information
docs_df = get_retrieved_documents(Client(), project_name=project_name)
docs_df["document_index"] = docs_df["document_metadata"].apply(lambda x: x["document_index"])
retrieved_doc_indices = docs_df.groupby("context.trace_id")["document_index"].apply(list)

# fetch the full input, output, context, and metadata for the traces
qa_df = get_input_output_context(Client(), project_name=project_name)
qa_df = qa_df.dropna(subset=["metadata"])
qa_df["expected_answer"] = qa_df["metadata"].apply(lambda x: x["expected_answer"])
qa_df["split"] = qa_df["metadata"].apply(lambda x: x["split"])
qa_df["expected_document_index"] = qa_df["metadata"].apply(lambda x: x["expected_document_index"])


# combine them
spans_df = (
    qa_df.reset_index()
    .merge(
        retrieved_doc_indices.rename("retrieved_document_indices"),
        left_on="context.trace_id",
        right_index=True,  # retrieved_doc_indices is indexed by context.trace_id
        how="left",
    )
    .set_index("context.span_id")
)
print(spans_df.shape)
spans_df.head()

## Define Evaluators


### 1. Refusal

Since we know some questions in our dataset are unanswerable, we expect the LLM to respond accordingly. So let's create an LLM judge to check for refusals in the output.


In [ ]:
from phoenix.evals import LLM, create_classifier

prompt = """
Given a question and response, determine if the response is a refusal to answer the question.

Question: {input}
Response: {output}

Is the response a refusal or an informative answer to the question?
"""

llm = LLM(model="gpt-4o", provider="openai")
refusal_evaluator = create_classifier(
    llm=llm,
    name="llm_refusal",
    prompt_template=prompt,
    choices={"refusal": 0, "answer": 1},
)

# test the evaluator on a single example
refusal_evaluator.evaluate(spans_df.iloc[0].to_dict())

### 2. Hallucination

Let's also check to see if our RAG pipeline is producing hallucinations. Phoenix evals has a built-in `HallucinationEvaluator` so we'll use that. First, let's inspect the `input_schema` so we know what it needs to run.


In [ ]:
from phoenix.evals import LLM
from phoenix.evals.metrics import HallucinationEvaluator

llm = LLM(model="gpt-4o", provider="openai")
hallucination_evaluator = HallucinationEvaluator(llm=llm)
hallucination_evaluator.describe()

Luckily, our data already has columns that match the `input_schema` for this evaluator. If that were not the case, we could provide an `input_mapping` so it works on our data and bind it to the evaluator so we can reuse it.


In [ ]:
# test the evaluator on a single example
hallucination_evaluator.evaluate(spans_df.iloc[0].to_dict())

### 3. Retrieval Precision

We also want to measure how well the information retrieval component of our system is working. Let's add a precision metric which checks to see how often the target document appeared in the retrieved results.


In [ ]:
from phoenix.evals import bind_evaluator, create_evaluator


@create_evaluator(name="precision")
def precision(retrieved_documents: list[int], relevant_documents: list[int]) -> float:
    relevant_set = set(relevant_documents)
    hits = sum(1 for doc in retrieved_documents if doc in relevant_set)
    return hits / len(retrieved_documents)


# our precision evaluator expects a list of relevant documents,
# but our dataset only has one relevant document per query, so we
# wrap the expected document index in a list inside our mapping using a lambda function
precision_mapping = {
    "relevant_documents": lambda x: [x["expected_document_index"]],
    "retrieved_documents": "retrieved_document_indices",
}

precision_evaluator = bind_evaluator(precision, precision_mapping)

# test the evaluator on a single example
precision_evaluator.evaluate(spans_df.iloc[0].to_dict())

### Putting it all together

Let's run our 3 evaluators on all of our project traces.


In [ ]:
from phoenix.evals import async_evaluate_dataframe

train_spans = spans_df[spans_df["split"] == "train"]
results = await async_evaluate_dataframe(
    dataframe=train_spans,
    evaluators=[precision_evaluator, hallucination_evaluator, refusal_evaluator],
    concurrency=10,
    exit_on_error=True,
)
results.head()

### Log trace evaluations back to Phoenix


In [ ]:
from phoenix.client import AsyncClient
from phoenix.evals.utils import to_annotation_dataframe

client = AsyncClient()

annotations = to_annotation_dataframe(
    dataframe=results
)  # can also specify score_names to log only certain scores
await client.spans.log_span_annotations_dataframe(dataframe=annotations)

# Improve Evaluators

Go into Phoenix and look at your project traces now that you've added some eval metrics. Pay attention to the "llm_refusal" metric - is it catching all the refusals?
No, it looks like it is not performing as expected.

Let's see if we can improve our LLM Judge so it is better aligned.

**Steps:**

1. Manually annotate some traces as "refused" or "responded" inside Phoenix.
2. Export those annotated traces and use to create a dataset for experimentation.
3. Define an LLM judge (refusal) and use as the experiment "task".
4. Create a simple heuristic experiment evaluator that checks for an exact match between the judge score and our annotation
5. Iterate on the judge prompt until we are happy with the results.

<center>
    <h3 align="left">Phoenix Experiments</h3>
    <p style="text-align:center">
        <img alt="eval lifecycle" src="https://storage.googleapis.com/arize-phoenix-assets/assets/images/experiment.png" width="1000"/>
    </p>
</center>


After manual annotation, pull down those traces:


In [ ]:
from phoenix.client import Client
from phoenix.client.types.spans import SpanQuery

# Export all the top level spans
query = SpanQuery().where("name == 'RetrieverQueryEngine.query'")
spans_df = Client().spans.get_spans_dataframe(query=query, project_identifier=project_name)

# Shape the spans dataframe
spans_df["query"] = spans_df["attributes.input.value"]
spans_df["response"] = spans_df["attributes.output.value"]
spans_df.dropna(subset=["attributes.metadata"], inplace=True)
spans_df["expected_answer"] = spans_df["attributes.metadata"].apply(lambda x: x["expected_answer"])

# Export annotations and add to the spans from earlier
annotations_df = Client().spans.get_span_annotations_dataframe(
    spans_dataframe=spans_df, project_identifier=project_name
)
refusal_ground_truth = annotations_df[
    (annotations_df["annotator_kind"] == "HUMAN") & (annotations_df["annotation_name"] == "refusal")
]
refusal_ground_truth = refusal_ground_truth.rename_axis(index={"span_id": "context.span_id"})
refusal_ground_truth = refusal_ground_truth.rename(columns={"result.score": "refusal_score"})
labeled_spans_df = spans_df.merge(
    refusal_ground_truth[["refusal_score"]], left_index=True, right_index=True, how="left"
)
labeled_spans_df = labeled_spans_df[
    ["context.span_id", "query", "response", "refusal_score", "expected_answer"]
]
labeled_spans = labeled_spans_df.dropna(subset=["refusal_score"])
labeled_spans["refusal_score"].value_counts()
print(labeled_spans["refusal_score"].value_counts())

In [ ]:
labeled_spans.head()

In [ ]:
dataset_name = "annotated-queries-train"
ds = await AsyncClient().datasets.create_dataset(
    name=dataset_name,
    dataframe=labeled_spans,
    input_keys=["query", "response"],
    output_keys=["refusal_score", "expected_answer"],
)

# get the dataset if already created
# ds = await AsyncClient().datasets.get_dataset(dataset=dataset_name)

Next, we define:

1. The experiment task (here, our LLM evaluator is the task).
2. The experiment evaluator (here, a simple check for alignment between the judge and human).

**Note: 2.0 evaluators are now drop-in compatible as experiment evaluators!**


In [ ]:
from phoenix.evals import create_evaluator


# experiment task
def run_refusal_eval(input, evaluator):
    result = evaluator.evaluate(input)
    return result[0]


# experiment evaluator
@create_evaluator(name="exact_match")
def exact_match(output, expected):
    return float(output["score"]) == float(expected["refusal_score"])

First, let's get a baseline by testing out the initial refusal evaluator against our annotations.


In [ ]:
prompt = """
Given a question and response, determine if the response is a refusal to answer the question.

Question: {query}
Response: {response}

Is the response a refusal or an informative response to the question?
"""

llm = LLM(model="gpt-4o", provider="openai")
baseline_refusal = create_classifier(
    llm=llm,
    name="refusal",
    prompt_template=prompt,
    choices={"refusal": 0, "answer": 1},
)

In [ ]:
from functools import partial

from phoenix.client import AsyncClient

async_client = AsyncClient()

experiment = await async_client.experiments.run_experiment(
    dataset=ds,
    task=partial(run_refusal_eval, evaluator=baseline_refusal),
    experiment_name="baseline",
    evaluators=[exact_match],
    concurrency=10,
    # dry_run=3,
)

Go to the experiment and filter by "exact_match==0" -- these are the examples where our refusal judge disagreed with the human annotations. Look at the RAG responses and the judge explanations. I notice that the LLM judge considers uncertain responses as valid answers rather than refusals.

Based on these observations, let's tweak our prompt to make the evaluation criteria more clear to the LLM judge. Describe exactly what a "refusal" looks like.


In [ ]:
from phoenix.evals import LLM, create_classifier

prompt = """
Given a question and response, determine if the response is a refusal to answer the question.
Refusals often contain phrases of uncertainty like 'I don't know' and 'I don't have that information'.
They also often mention that the answer is not provided in the information or context.

If the response contains these phrases, it is a refusal. Even if the response contains other
text indicating an attempt to answer the question, it is still a refusal.

If the response does not contain these "hedging" phrases, it is an informative response. Do not
consider the correctness of the response, only whether it is a refusal or not.

Question: {query}
Response: {response}

Is the response a refusal or an informative answer to the question?
"""

refusal_v2 = create_classifier(
    llm=llm,
    name="refusal",
    prompt_template=prompt,
    choices={"refusal": 0, "answer": 1},
)

In [ ]:
experiment = await async_client.experiments.run_experiment(
    dataset=ds,
    task=partial(run_refusal_eval, evaluator=refusal_v2),
    experiment_name="prompt-v2",
    evaluators=[exact_match],
    concurrency=10,
    # dry_run=3,
)

Looking at this experiment in Phoenix, I see that we now have "exact_match == 1.0" indicating 100% agreement between our new judge and the annotations!

Through experimentation we were able to improve the evaluation metric itself, much in the same way we would improve any process.


# Improve the Application

Now that we feel good about our refusal metric, let's see if we can improve our RAG system.

Exactly 50% of the queries in our dataset are unanswerable, so ideally we would like to see the "llm_refusal" score close to 0.5. We don't want the RAG system attempting to answer questions that are not answerable from the context because this increases the chances of hallucination - not good!

**Steps:**

1. Create a dataset using the train set queries.
2. Define our experiment task (running RAG on our dataset).
3. Use our new and improved refusal classifier as the experiment evaluator.
4. Iterate on the RAG agent's prompt until we are happy.


In [ ]:
dataset_name = "train-queries"
ds = await AsyncClient().datasets.create_dataset(
    name=dataset_name,
    dataframe=train_queries,
    input_keys=["query"],
)

# if already created
# ds = await AsyncClient().datasets.get_dataset(dataset=dataset_name)

In [ ]:
from phoenix.evals import bind_evaluator


# define experiment task (running the RAG engine)
async def run_rag_task(input, rag_engine):
    """Ask a question of the knowledge base."""
    response = rag_engine.query(input["query"])
    return response


# use an input mapping to fit our dataset to the evaluator we created earlier
refusal_evaluator = bind_evaluator(refusal_v2, {"query": "input.query", "response": "output"})

### Experiment 1: Baseline RAG System

Let's rerun our initial RAG system to get a baseline. How do the "out-of-the-box" defaults work?


In [ ]:
query_engine_baseline = index.as_query_engine()
baseline_experiment = await AsyncClient().experiments.run_experiment(
    dataset=ds,
    task=partial(run_rag_task, rag_engine=query_engine_baseline),
    experiment_name="baseline",
    evaluators=[refusal_evaluator],
    concurrency=10,
    # dry_run=3,
)

### Experiment 2: RAG with Custom Prompt

Go into Phoenix to see the results of our experiment.

The refusal score is a little high - we want to get it down closer to 0.5 since we know 50% of our queries are unanswerable. Let's see if modifying the system prompt used for the LLM generation component of our RAG system helps.


In [ ]:
from textwrap import dedent

custom_system_prompt = """You are an expert at answering questions about a given context.
\nAlways answer the query using the provided context information, and not prior knowledge.
\nSome rules to follow:
\n1. Never directly reference the given context in your answer.
\n2. Avoid statements like 'Based on the context, ...' or 'The context information ...'
or anything along those lines.
\n3. Do NOT use prior knowledge to answer the question. Only use the context provided.
\n4. If you cannot find the answer in the context, say 'I cannot find that information.' When in
doubt, default to responding 'I cannot find that information.'
"""
custom_query_engine = index.as_query_engine(system_prompt=dedent(custom_system_prompt))

In [ ]:
experiment = await AsyncClient().experiments.run_experiment(
    dataset=ds,
    task=partial(run_rag_task, rag_engine=custom_query_engine),
    experiment_name="custom-prompt",
    evaluators=[refusal_evaluator],
    concurrency=10,
    # dry_run=3,
)

Check out the results of this experiment in Phoenix.

Nice, we are heading in the right direction! Our refusal score went down a bit closer to 0.5, indicating that our RAG system is correctly refusing to answer more queries.


# Conclusion


In this notebook, we have covered a lot! Now you know:

1. How to evaluate traces using different types of evaluators:
   - custom LLM classifiers
   - built-in metrics
   - heuristic functions using the `create_evaluator` decorator
2. How to build and iterate on an LLM Evaluator using experiments
3. How to iterate on an application using experiments and evaluators

For more information, check out our [Documentation!](https://arize-phoenix.readthedocs.io/projects/evals/en/latest/)
